# Batch-wise Relaxations

In this tutorial, we show how to use the TorchStructureLBFGS. Latter allows for relaxing structures in a batch-wise manner, i.h. it optimizes multiple structures in parallel. This is particularely useful, when many relatively similar structures (--> similar time until convergence) should be relaxed while requiring possibly short simulation time.

In [ ]:
import torch
from ase.io import read

import schnetpack as spk
from schnetpack import properties
from schnetpack.interfaces.ase_interface import AtomsConverter
from schnetpack.interfaces.batchwise_optimizer import TorchStructureLBFGS

First, we load the force field model that provides the forces for the relaxation process. The avoid backwarding through the graph multiple times, the respond module (forces module) is disabled.

In [ ]:
model_path = "/home/jonas/Documents/schnetpack/tests/testdata/md_ethanol.model"

# set device
device = torch.device("cuda")

# load model
model = torch.load(model_path, map_location=device)

# remove response modules (to avoid differentiating more than once)
model.model_outputs = ["energy"]
model.do_postprocessing = False
model.required_derivatives = []
model.output_modules = torch.nn.ModuleList([model.output_modules[0]])

Subsequently, we load the batch of initial structures utilizing ASE (supports xyz, db and more) and convert it to SchNetPack input format. For this purpose we need use the AtomsConverter with suitable neighbor list. 

In [ ]:
input_structure_file = "/home/jonas/Documents/schnetpack/tests/testdata/md_ethanol.xyz"

# load initial structures
ats = read(input_structure_file, index=":")

# define neighbor list
cutoff = model.representation.cutoff.item()
nbh_list=spk.transform.MatScipyNeighborList(cutoff=cutoff)

# build atoms converter
atoms_converter = AtomsConverter(
    neighbor_list=nbh_list,
    device=device,
)

# convert atoms object to schnetpack batch
inputs = atoms_converter(ats)

For some systems it helps to fix the positions of certain atoms during the relaxation. In TorchStructureLBFGS this can be achieved by providing a mask of boolean entries. The mask is a list of n_atoms * 3 entries, i.h. it contains 3 entries for each atom associated with the respective directions in cartesian space. Here we don't fix any atoms. Hence, the mask only contains True values.

In [ ]:
# define structure mask for optimization (True for fixed, False for non-fixed)
n_atoms = len(ats[0].get_atomic_numbers())
single_structure_mask = [False for _ in range(n_atoms * 3)]
# expand mask by number of input structures (fixed atoms are equivalent for all input structures)
mask = single_structure_mask * len(ats)

Finally, we run the optimization

In [ ]:
# Initialize optimizer
optimizer = TorchStructureLBFGS(
    model=model,
    model_inputs=inputs,
    fixed_atoms_mask=mask
)

# run optimization
optimizer.run(fmax=0.0005, max_opt_steps=1000)

In [ ]:
# get new atomic positions
optimizer.get_relaxed_structure()